In [14]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [15]:
spammer_order = "../feature.json"
raw_file = pd.read_json(open(spammer_order), orient='records')
raw_file.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1214054254,0,0,0,15,0.500000,0,0,0,1005,...,0,7,0,0,0.034592,0,1,30972.857143,8.239935e+08,no
1,2158838150,0,0,0,10,1.733333,0,0,0,1819,...,0,3,0,0,0.026582,0,0,33023.000000,7.491242e+08,no
2,1465038162,0,0,0,18,0.984615,0,0,0,3542,...,0,3,0,0,0.027306,0,2,36454.250000,6.229687e+08,no
3,5203407567,0,0,0,27,0.012920,0,0,0,340,...,0,19,0,0,0.031989,0,4,11511.346939,7.102748e+08,no
4,3944654540,0,0,0,7,0.204396,0,0,0,371,...,0,3,0,0,0.055030,0,38,26378.770833,7.995144e+08,yes


In [16]:
all_feat_list = ['figure_RRT', 'average_comm', 'figure_at_sigle', 'hashtag', 'reputation', 
                 'late_night_times', 'followee_num', 'follow_ratio', 'post_num', 'figure_at', 
                 'active_day_ratio', 'mention', 'day_interval_variance', 'figure_at_every', 
                 'test_similarity', 'text_similarity', 'url', 'time_interval_mean', 'time_interval_var']

idx_title = ['id'] + all_feat_list + ['is_spammer']
print(idx_title)
raw_file.rename(columns={i: idx_title[i] for i in range(21)}, inplace=True)

['id', 'figure_RRT', 'average_comm', 'figure_at_sigle', 'hashtag', 'reputation', 'late_night_times', 'followee_num', 'follow_ratio', 'post_num', 'figure_at', 'active_day_ratio', 'mention', 'day_interval_variance', 'figure_at_every', 'test_similarity', 'text_similarity', 'url', 'time_interval_mean', 'time_interval_var', 'is_spammer']


In [17]:
raw_file.head()

,id,figure_RRT,average_comm,figure_at_sigle,hashtag,reputation,late_night_times,followee_num,follow_ratio,post_num,...,active_day_ratio,mention,day_interval_variance,figure_at_every,test_similarity,text_similarity,url,time_interval_mean,time_interval_var,is_spammer
0,1214054254,0,0,0,15,0.500000,0,0,0,1005,...,0,7,0,0,0.034592,0,1,30972.857143,8.239935e+08,no
1,2158838150,0,0,0,10,1.733333,0,0,0,1819,...,0,3,0,0,0.026582,0,0,33023.000000,7.491242e+08,no
2,1465038162,0,0,0,18,0.984615,0,0,0,3542,...,0,3,0,0,0.027306,0,2,36454.250000,6.229687e+08,no
3,5203407567,0,0,0,27,0.012920,0,0,0,340,...,0,19,0,0,0.031989,0,4,11511.346939,7.102748e+08,no
4,3944654540,0,0,0,7,0.204396,0,0,0,371,...,0,3,0,0,0.055030,0,38,26378.770833,7.995144e+08,yes


In [8]:
raw_file.describe()

,id,figure_RRT,average_comm,figure_at_sigle,hashtag,reputation,late_night_times,followee_num,follow_ratio,post_num,figure_at,active_day_ratio,mention,day_interval_variance,figure_at_every,test_similarity,text_similarity,url,time_interval_mean,time_interval_var
count,1.566000e+03,1566.0,1566.0,1566.0,1566.000000,1566.000000,1566.0,1566.0,1566.0,1566.000000,1566.0,1566.0,1566.000000,1566.0,1566.0,1566.000000,1566.0,1566.000000,1566.000000,1.566000e+03
mean,2.967337e+09,0.0,0.0,0.0,10.814815,2.009261,0.0,0.0,0.0,1460.061941,0.0,0.0,6.033206,0.0,0.0,0.062515,0.0,7.306513,29051.849704,7.228954e+08
std,1.184598e+09,0.0,0.0,0.0,9.773090,28.911290,0.0,0.0,0.0,3472.917546,0.0,0.0,7.110370,0.0,0.0,0.091577,0.0,10.694027,10882.851825,2.586876e+08
min,1.005384e+09,0.0,0.0,0.0,0.000000,0.009146,0.0,0.0,0.0,2.000000,0.0,0.0,0.000000,0.0,0.0,0.020051,0.0,0.000000,6.204082,0.000000e+00
25%,2.021067e+09,0.0,0.0,0.0,3.000000,0.317523,0.0,0.0,0.0,294.750000,0.0,0.0,1.000000,0.0,0.0,0.030094,0.0,1.000000,22165.185566,6.211617e+08
50%,2.727800e+09,0.0,0.0,0.0,8.000000,0.579435,0.0,0.0,0.0,669.000000,0.0,0.0,4.000000,0.0,0.0,0.037440,0.0,3.000000,30547.811437,7.618443e+08
75%,3.540640e+09,0.0,0.0,0.0,16.000000,1.051232,0.0,0.0,0.0,1520.750000,0.0,0.0,8.000000,0.0,0.0,0.055187,0.0,8.000000,37380.515306,8.798472e+08
max,5.397886e+09,0.0,0.0,0.0,49.000000,1129.000000,0.0,0.0,0.0,102353.000000,0.0,0.0,50.000000,0.0,0.0,1.000000,0.0,50.000000,65396.666667,1.731597e+09


In [27]:
aval_feat_list = ['hashtag', 'reputation', 'post_num', 'mention', 'test_similarity', 'url', 'time_interval_mean',
                 'time_interval_var']
data = raw_file[aval_feat_list]
target = raw_file[['is_spammer']]

In [19]:
# split to train and test sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    data, target, test_size=0.33, random_state=0)

In [20]:
train_X = train_X.as_matrix()
test_X = test_X.as_matrix()
train_y = train_y.as_matrix()

In [21]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
gbm.fit(train_X, train_y)
predictions = gbm.predict(test_X)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.41%


In [24]:
# or use 10 fold cross validation
from sklearn.model_selection import cross_val_score
model = xgb.XGBClassifier()

# from sklearn.model_selection import KFold
# kfold = KFold(n_splits=7)

from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

results = cross_val_score(model, data, target, cv=cv)
print(results)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# initial mean accuracy 48.54%, deviation 31.44%

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

[ 0.72978723  0.72765957  0.71702128]
Accuracy: 72.48% (0.56%)


## start adjusting the xgboost model parameters

In [25]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import ShuffleSplit
    

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [26]:
data = pd.read_csv('spammer_order.csv')

FileNotFoundError: File b'spammer_order.csv' does not exist

In [ ]:
data.shape

In [ ]:
feat_list = ['post_num', 'follower_num', 'followee_num', 'content_similar',
 'figure_jing', 'figure_url', 'figure_face', 'figure_RRT', 'figure_face_every',
 'figure_jing_every', 'figure_url_every', 'figure_url_single', 'figure_jing_single',
 'figure_at', 'figure_at_every', 'figure_at_single', 'average_repost', 'average_comm',
 'late_night_times', 'is_regular', 'shorttime_times', 'active_day_ratio', 'day_interval_variance',
 'day_in_variance', 'follow_ratio']
target='is_spammer'

In [ ]:
data[target] = data[target].map(dict(yes=1, no=0))
data[target].value_counts()

In [ ]:
def modelfit(alg, dtrain, cv_folds=5, early_stopping_rounds=50):
    
    xgb_param = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(dtrain[feat_list].values, label=dtrain[target].values)
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
        metrics='auc', early_stopping_rounds=early_stopping_rounds)
    alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[feat_list], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[feat_list])
    dtrain_predprob = alg.predict_proba(dtrain[feat_list])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    

In [ ]:
def cross_val(alg, data):
#     kfold = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
    kfold = KFold(n_splits=10, random_state=0)
    train = data[feat_list]
    res = data[target]
    scoring = ['accuracy', 'recall']
    results = cross_val_score(alg, train, res, cv=kfold, scoring='accuracy')
    print results
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    # initial mean accuracy 48.54%, deviation 31.44%

## 1. Find the number of estimators for a high learning rate

In [ ]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)

# from sklearn import svm
# clf = svm.SVC()
# cross_val(clf, data)

modelfit(xgb1, data)

# cross_val(xgb1, data)

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}

from sklearn.model_selection import ShuffleSplit
shuffle = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=True, cv=shuffle)
gsearch1.fit(data[feat_list],data[target])

In [ ]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[6,7,8],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(data[feat_list],data[target])

In [ ]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test2b = {
    'min_child_weight':[6,8,10,12, 14, 16]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=7,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(data[feat_list],data[target])

In [ ]:
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

In [ ]:
param_test2c = {
    'min_child_weight':[16, 20, 24, 28, 32, 36, 40]
}
gsearch2c = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=7,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2c, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2c.fit(data[feat_list],data[target])

In [ ]:
gsearch2c.grid_scores_, gsearch2c.best_params_, gsearch2c.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(data[feat_list],data[target])

In [ ]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=7,
        min_child_weight=36,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
# modelfit(xgb2, data)
cross_val(xgb2, data)

### Tune subsample and colsample_bytree

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=7,
                                        min_child_weight=36, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(data[feat_list],data[target])

In [ ]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

### regularization

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=7,
                                        min_child_weight=36, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(data[feat_list],data[target])

In [ ]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test7 = {
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=7,
                                        min_child_weight=36, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(data[feat_list],data[target])

In [ ]:
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

In [ ]:
xgb3 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=7,
        min_child_weight=36,
        gamma=0.1,
        subsample=0.7,
        colsample_bytree=0.8,
        reg_alpha=0.01,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb3, data)

In [ ]:
cross_val(xgb3, data)